## Requirements
- The XRADIO images are stored xr.Dataset that remains compatible with Xarray
    - Xarray API should work with structure (to_zarr, open_dataset, open_zarr)
        - No numpy data structures in attributes (otherwise serialization fails). Use list and standard numerical types.
        - No other xarray data structures in attributes (look into storing history as just a dict).
    - Allow multiple images and image types as long as they share coordinates.
- Make use of the same convention as Measurement Set Module/Schema
    - `read_image` renamed to `open_image`
    - All measures should be formatted as Xarray-compatible dicts. For example:
    ```Python
        reference_direction = {'attrs': {'frame': 'fk5',
                                'type': 'sky_coord',
                                'units': 'rad',
                                'equinox': 'j2000.0'},
            'data': [3.5392577860590637, 0.5324852164475642],
            'dims': "sky_dir_label",
            'coords': {"sky_dir_label":
                            {"data":["ra","dec"]
                            "dims":"sky_dir_label"}
                    }
        }
    ```
    - Try reusing the coordinate schemas from the measurement set schema (for example, frequency coordinate).
    - Add frame to location called native_projection for native_pole_direction (latpole and lonpole in fits). latpole and lonpole is combined in a single measure called native_pole_direction.
    - Renamed BEAM to BEAM_FIT_PARAMS and beam_param to beam_param_label.
    - Add types to all data variables and dataset.

## Image Types
    - SKY
    - MASK_SKY (internal mask in CASA)
    - VISIBILITY*
    - VISIBILITY_NORMALIZATION
    - MODEL
    - RESIDUAL
    - MASK_RESIDUAL (internal mask in CASA)
    - POINT_SPREAD_FUNCTION
    - UV_SAMPLING*
    - UV_SAMPLING_NORMALIZATION (sensitivity, sum of weights)
    - PRIMARY_BEAM
    - APERTURE*
    - APERTURE_NORMALIZATION*
    - MASK
    - BEAM_FIT_PARAMS

* UV plain. Only used internally and for debugging.
    
## Questions:
- What is the use case for multiple rest_frequencies (this only seems to be a thing for FITS images)?
- If the addition of datagroups, can we depricate active_mask?
- How to handle the different types of masks:
    - External Mask added as just another image.
    - Internal Masks are currently added by appending the image they come from. For example, MASK_RESIDUAL, MASK_SKY.

## Issues
- Visibility normalization is not loaded correctly (sum_weight). Adds unnecessary l and m coordinates and tiles. 

## Measures example

In [ ]:
import xarray as xr 

reference_direction = {'attrs': {'frame': 'fk5',
                        'type': 'sky_coord',
                        'units': 'rad',
                        'equinox': 'j2000.0'},
    'data': [3.5392577860590637, 0.5324852164475642],
    'dims': "sky_dir_label",
    'coords': {"sky_dir_label":
                    {"data":["ra","dec"],
                     "dims":"sky_dir_label"}
            }
}

dir_xda = xr.DataArray.from_dict(reference_direction)
dir_xda

## Download Data

In [7]:
import toolviper

toolviper.utils.data.update()
toolviper.utils.data.download("3c286_Band6_5chans_lsrk_robust_0.5_niter_99_fits")
toolviper.utils.data.download("3c286_Band6_5chans_lsrk_robust_0.5_niter_99_casa")

casa_dir = "3c286_Band6_5chans_lsrk_robust_0.5_niter_99_casa/"
fits_dir = "3c286_Band6_5chans_lsrk_robust_0.5_niter_99_fits/"

[2025-12-11 08:32:36,521]     INFO  graphviper:  Updating file metadata information ...  


Output()

[2025-12-11 08:32:36,665]     INFO  graphviper:  Downloading from [cloudflare] .... 


 Download List                                     
 ────────────────────────────────────────────────── 
  3c286_Band6_5chans_lsrk_robust_0.5_niter_99_fits

[2025-12-11 08:32:36,668]     INFO  graphviper:  File exists: 3c286_Band6_5chans_lsrk_robust_0.5_niter_99_fits 


Output()

[2025-12-11 08:32:36,671]     INFO  graphviper:  Downloading from [cloudflare] .... 


 Download List                                     
 ────────────────────────────────────────────────── 
  3c286_Band6_5chans_lsrk_robust_0.5_niter_99_casa

[2025-12-11 08:32:36,673]     INFO  graphviper:  File exists: 3c286_Band6_5chans_lsrk_robust_0.5_niter_99_casa 


Output()

# Load and Open a Single Image

In [8]:
#Loading a single casa or fits image into an xarray Dataset

from graphviper.graph_tools.coordinate_utils import make_parallel_coord
from toolviper.utils.display import dict_to_html
from IPython.display import HTML, display

from xradio.image import open_image, load_image
import toolviper.utils.logger as logger

img_xds = open_image(
        fits_dir + "3c286_Band6_5chans_lsrk_robust_0.5_niter_99.image.fits",
)

img_xds = open_image(
        casa_dir + "3c286_Band6_5chans_lsrk_robust_0.5_niter_99.image",
)

img_xds

Successful readonly open of default-locked table 3c286_Band6_5chans_lsrk_robust_0.5_niter_99_casa/3c286_Band6_5chans_lsrk_robust_0.5_niter_99.image: 1 columns, 1 rows


<xarray.Dataset> Size: 7MB
Dimensions:            (time: 1, frequency: 5, polarization: 4, l: 250, m: 250,
                        beam_params_label: 3)
Coordinates:
  * time               (time) float64 8B 5.684e+04
  * frequency          (frequency) float64 40B 2.398e+11 2.399e+11 ... 2.403e+11
  * polarization       (polarization) <U1 16B 'I' 'Q' 'U' 'V'
  * l                  (l) float64 2kB 6.06e-05 6.012e-05 ... -6.012e-05
  * m                  (m) float64 2kB -6.06e-05 -6.012e-05 ... 6.012e-05
  * beam_params_label  (beam_params_label) <U5 60B 'major' 'minor' 'pa'
    velocity           (frequency) float64 40B 2.216e+05 6.54e+04 ... -4.03e+05
    right_ascension    (l, m) float64 500kB 3.539 3.539 3.539 ... 3.539 3.539
    declination        (l, m) float64 500kB 0.5324 0.5324 ... 0.5325 0.5325
Data variables:
    SKY                (time, frequency, polarization, l, m) float32 5MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    BEAM_FIT_PARAMS    (time, frequency, polarization, beam_params_label) float64 480B dask.array<chunksize=(1, 5, 4, 3), meta=np.ndarray>
    MASK_SKY           (time, frequency, polarization, l, m) bool 1MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
Attributes:
    coordinate_system_info:  {'reference_direction': {'attrs': {'frame': 'fk5...
    type:                    image
    data_groups:             {'base': {'beam_fit_params': 'BEAM_FIT_PARAMS', ...

In [9]:
display(HTML(dict_to_html(img_xds.attrs)))

In [10]:
display(HTML(dict_to_html(img_xds.SKY.attrs)))

In [11]:
#Loading a single casa or fits image into an xarray Dataset

from graphviper.graph_tools.coordinate_utils import make_parallel_coord
from toolviper.utils.display import dict_to_html
from IPython.display import HTML, display

from xradio.image import open_image, load_image
import toolviper.utils.logger as logger


img_xds = load_image(
        casa_dir + "3c286_Band6_5chans_lsrk_robust_0.5_niter_99.image",
)

img_xds

Successful readonly open of default-locked table 3c286_Band6_5chans_lsrk_robust_0.5_niter_99_casa/3c286_Band6_5chans_lsrk_robust_0.5_niter_99.image: 1 columns, 1 rows


<xarray.Dataset> Size: 7MB
Dimensions:            (time: 1, frequency: 5, polarization: 4, l: 250, m: 250,
                        beam_params_label: 3)
Coordinates:
  * time               (time) float64 8B 5.684e+04
  * frequency          (frequency) float64 40B 2.398e+11 2.399e+11 ... 2.403e+11
  * polarization       (polarization) <U1 16B 'I' 'Q' 'U' 'V'
  * l                  (l) float64 2kB 6.06e-05 6.012e-05 ... -6.012e-05
  * m                  (m) float64 2kB -6.06e-05 -6.012e-05 ... 6.012e-05
  * beam_params_label  (beam_params_label) <U5 60B 'major' 'minor' 'pa'
    velocity           (frequency) float64 40B 2.216e+05 6.54e+04 ... -4.03e+05
    right_ascension    (l, m) float64 500kB 3.539 3.539 3.539 ... 3.539 3.539
    declination        (l, m) float64 500kB 0.5324 0.5324 ... 0.5325 0.5325
Data variables:
    SKY                (time, frequency, polarization, l, m) float32 5MB -0.0...
    BEAM_FIT_PARAMS    (time, frequency, polarization, beam_params_label) float64 480B ...
    MASK_SKY           (time, frequency, polarization, l, m) bool 1MB False ....
Attributes:
    coordinate_system_info:  {'reference_direction': {'attrs': {'frame': 'fk5...
    type:                    image
    data_groups:             {'base': {'beam_fit_params': 'BEAM_FIT_PARAMS', ...

## Loading Mutiple CASA and FITS Images

- VISIBILITY_NORMALIZATION is incorrectly loaded.
- Can mix FITS and CASA images (still have to share coordinates and have the same Dataset level attributes). 

In [13]:
from xradio.image import open_image
img_xds = open_image(    
    {"sky": casa_dir + "3c286_Band6_5chans_lsrk_robust_0.5_niter_99.image",
    "model": fits_dir + "3c286_Band6_5chans_lsrk_robust_0.5_niter_99.model.fits",
    "residual": casa_dir + "3c286_Band6_5chans_lsrk_robust_0.5_niter_99.residual",
    "mask": casa_dir + "3c286_Band6_5chans_lsrk_robust_0.5_niter_99.mask",
    "primary_beam": fits_dir + "3c286_Band6_5chans_lsrk_robust_0.5_niter_99.pb.fits",
    "point_spread_function": casa_dir + "3c286_Band6_5chans_lsrk_robust_0.5_niter_99.psf",
    "visibility_normalization": casa_dir + "3c286_Band6_5chans_lsrk_robust_0.5_niter_99.sumwt"})

img_xds

Successful readonly open of default-locked table 3c286_Band6_5chans_lsrk_robust_0.5_niter_99_casa/3c286_Band6_5chans_lsrk_robust_0.5_niter_99.image: 1 columns, 1 rows
Successful readonly open of default-locked table 3c286_Band6_5chans_lsrk_robust_0.5_niter_99_casa/3c286_Band6_5chans_lsrk_robust_0.5_niter_99.residual: 1 columns, 1 rows
Successful readonly open of default-locked table 3c286_Band6_5chans_lsrk_robust_0.5_niter_99_casa/3c286_Band6_5chans_lsrk_robust_0.5_niter_99.mask: 1 columns, 1 rows
Successful readonly open of default-locked table 3c286_Band6_5chans_lsrk_robust_0.5_niter_99_casa/3c286_Band6_5chans_lsrk_robust_0.5_niter_99.psf: 1 columns, 1 rows
Successful readonly open of default-locked table 3c286_Band6_5chans_lsrk_robust_0.5_niter_99_casa/3c286_Band6_5chans_lsrk_robust_0.5_niter_99.sumwt: 1 columns, 1 rows


/Users/jsteeb/miniforge3/envs/timg/lib/python3.13/site-packages/dask/array/core.py:4997: PerformanceWarning: Increasing number of chunks by factor of 250
  result = blockwise(


<xarray.Dataset> Size: 39MB
Dimensions:                   (time: 1, frequency: 5, polarization: 4, l: 250,
                               m: 250, beam_params_label: 3)
Coordinates:
  * time                      (time) float64 8B 5.684e+04
  * frequency                 (frequency) float64 40B 2.398e+11 ... 2.403e+11
  * polarization              (polarization) <U1 16B 'I' 'Q' 'U' 'V'
  * l                         (l) float64 2kB 6.06e-05 6.012e-05 ... -6.012e-05
  * m                         (m) float64 2kB -6.06e-05 -6.012e-05 ... 6.012e-05
  * beam_params_label         (beam_params_label) <U5 60B 'major' 'minor' 'pa'
    velocity                  (frequency) float64 40B 2.216e+05 ... -4.03e+05
    right_ascension           (l, m) float64 500kB 3.539 3.539 ... 3.539 3.539
    declination               (l, m) float64 500kB 0.5324 0.5324 ... 0.5325
Data variables:
    SKY                       (time, frequency, polarization, l, m) float32 5MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    BEAM_FIT_PARAMS           (time, frequency, polarization, beam_params_label) float64 480B dask.array<chunksize=(1, 5, 4, 3), meta=np.ndarray>
    MASK_SKY                  (time, frequency, polarization, l, m) bool 1MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    MODEL                     (time, frequency, polarization, l, m) float32 5MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    RESIDUAL                  (time, frequency, polarization, l, m) float32 5MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    MASK_RESIDUAL             (time, frequency, polarization, l, m) bool 1MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    MASK                      (time, frequency, polarization, l, m) float32 5MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    PRIMARY_BEAM              (time, frequency, polarization, l, m) float32 5MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    POINT_SPREAD_FUNCTION     (time, frequency, polarization, l, m) float32 5MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    VISIBILITY_NORMALIZATION  (time, frequency, polarization, l, m) float32 5MB dask.array<chunksize=(1, 5, 4, 1, 1), meta=np.ndarray>
Attributes:
    coordinate_system_info:  {'reference_direction': {'attrs': {'frame': 'fk5...
    type:                    image
    data_groups:             {'base': {'beam_fit_params': 'BEAM_FIT_PARAMS', ...

## Writing to Zarr

In [16]:
import xarray as xr
if "VISIBILITY_NORMALIZATION" in img_xds.data_vars:
    del img_xds["VISIBILITY_NORMALIZATION"] # remove incorrectly loaded dataarray
img_xds.to_zarr("test_zarr_write.zarr", mode="w")
img_write_test_open = xr.open_zarr("test_zarr_write.zarr")
img_write_test_open

<xarray.Dataset> Size: 34MB
Dimensions:                (time: 1, frequency: 5, polarization: 4,
                            beam_params_label: 3, l: 250, m: 250)
Coordinates:
  * time                   (time) float64 8B 5.684e+04
  * frequency              (frequency) float64 40B 2.398e+11 ... 2.403e+11
  * polarization           (polarization) <U1 16B 'I' 'Q' 'U' 'V'
  * beam_params_label      (beam_params_label) <U5 60B 'major' 'minor' 'pa'
  * l                      (l) float64 2kB 6.06e-05 6.012e-05 ... -6.012e-05
  * m                      (m) float64 2kB -6.06e-05 -6.012e-05 ... 6.012e-05
    declination            (l, m) float64 500kB dask.array<chunksize=(125, 250), meta=np.ndarray>
    right_ascension        (l, m) float64 500kB dask.array<chunksize=(125, 250), meta=np.ndarray>
    velocity               (frequency) float64 40B dask.array<chunksize=(5,), meta=np.ndarray>
Data variables:
    BEAM_FIT_PARAMS        (time, frequency, polarization, beam_params_label) float64 480B dask.array<chunksize=(1, 5, 4, 3), meta=np.ndarray>
    MASK                   (time, frequency, polarization, l, m) float32 5MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    MASK_RESIDUAL          (time, frequency, polarization, l, m) bool 1MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    MASK_SKY               (time, frequency, polarization, l, m) bool 1MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    MODEL                  (time, frequency, polarization, l, m) float32 5MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    POINT_SPREAD_FUNCTION  (time, frequency, polarization, l, m) float32 5MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    PRIMARY_BEAM           (time, frequency, polarization, l, m) float32 5MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    RESIDUAL               (time, frequency, polarization, l, m) float32 5MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    SKY                    (time, frequency, polarization, l, m) float32 5MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
Attributes:
    coordinate_system_info:  {'native_pole_direction': {'attrs': {'frame': 'N...
    data_groups:             {'base': {'beam_fit_params': 'BEAM_FIT_PARAMS', ...
    type:                    image

## Writing to CASA

- Need to implement a better convention for file extensions.

In [17]:
from xradio.image import open_image, write_image
write_image(img_xds, "test_casa_write", overwrite=True)
img_xds_written = open_image("test_casa_write.sky")
img_xds_written

Successful readonly open of default-locked table test_casa_write.sky: 1 columns, 1 rows


<xarray.Dataset> Size: 7MB
Dimensions:            (time: 1, frequency: 5, polarization: 4, l: 250, m: 250,
                        beam_params_label: 3)
Coordinates:
  * time               (time) float64 8B 5.684e+04
  * frequency          (frequency) float64 40B 2.398e+11 2.399e+11 ... 2.403e+11
  * polarization       (polarization) <U1 16B 'I' 'Q' 'U' 'V'
  * l                  (l) float64 2kB 6.06e-05 6.012e-05 ... -6.012e-05
  * m                  (m) float64 2kB -6.06e-05 -6.012e-05 ... 6.012e-05
  * beam_params_label  (beam_params_label) <U5 60B 'major' 'minor' 'pa'
    velocity           (frequency) float64 40B 2.216e+05 6.54e+04 ... -4.03e+05
    right_ascension    (l, m) float64 500kB 3.539 3.539 3.539 ... 3.539 3.539
    declination        (l, m) float64 500kB 0.5324 0.5324 ... 0.5325 0.5325
Data variables:
    SKY                (time, frequency, polarization, l, m) float32 5MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    BEAM_FIT_PARAMS    (time, frequency, polarization, beam_params_label) float64 480B dask.array<chunksize=(1, 5, 4, 3), meta=np.ndarray>
    MASK_SKY           (time, frequency, polarization, l, m) bool 1MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
Attributes:
    coordinate_system_info:  {'reference_direction': {'attrs': {'frame': 'fk5...
    type:                    image
    data_groups:             {'base': {'beam_fit_params': 'BEAM_FIT_PARAMS', ...

## Create an empty image

In [18]:
from xradio.image import make_empty_sky_image

phase_center = img_xds.attrs['coordinate_system_info']['reference_direction']['data']
image_shape = img_xds.SKY.shape
cell_size = [(img_xds.l[1]-img_xds.l[0]).values.item(), (img_xds.m[1]-img_xds.m[0]).values.item()]
frequency_coords = img_xds.frequency.values.tolist()
pol_coords = img_xds.polarization.values.tolist()
time_coords = img_xds.time.values.tolist()

empty_img_xds = make_empty_sky_image(phase_center=phase_center,
    image_size=image_shape[-2:],
    cell_size=cell_size,
    frequency_coords=frequency_coords,
    pol_coords=pol_coords,
    time_coords=time_coords,
    direction_reference = "fK5",
    projection = "SIN",
    spectral_reference = "lsrk",
    do_sky_coords = True,
)

empty_img_xds

<xarray.Dataset> Size: 1MB
Dimensions:            (time: 1, frequency: 5, polarization: 4, l: 250, m: 250,
                        beam_params_label: 3)
Coordinates:
  * time               (time) float64 8B 5.684e+04
  * frequency          (frequency) float64 40B 2.398e+11 2.399e+11 ... 2.403e+11
  * polarization       (polarization) <U1 16B 'I' 'Q' 'U' 'V'
  * l                  (l) float64 2kB 6.06e-05 6.012e-05 ... -6.012e-05
  * m                  (m) float64 2kB -6.06e-05 -6.012e-05 ... 6.012e-05
  * beam_params_label  (beam_params_label) <U5 60B 'major' 'minor' 'pa'
    velocity           (frequency) float64 40B 3.122e+05 ... -3.122e+05
    right_ascension    (l, m) float64 500kB 3.539 3.539 3.539 ... 3.539 3.539
    declination        (l, m) float64 500kB 0.5324 0.5324 ... 0.5325 0.5325
Data variables:
    *empty*
Attributes:
    data_groups:             {'base': {}}
    coordinate_system_info:  {'reference_direction': {'attrs': {'frame': 'fk5...
    type:                    image

## Image Accessor

In [21]:
#Loading a single casa or fits image into an xarray Dataset
from xradio.image import open_image


img_xds = open_image(
        casa_dir + "3c286_Band6_5chans_lsrk_robust_0.5_niter_99.image",
)

img_xds

Successful readonly open of default-locked table 3c286_Band6_5chans_lsrk_robust_0.5_niter_99_casa/3c286_Band6_5chans_lsrk_robust_0.5_niter_99.image: 1 columns, 1 rows


<xarray.Dataset> Size: 7MB
Dimensions:            (time: 1, frequency: 5, polarization: 4, l: 250, m: 250,
                        beam_params_label: 3)
Coordinates:
  * time               (time) float64 8B 5.684e+04
  * frequency          (frequency) float64 40B 2.398e+11 2.399e+11 ... 2.403e+11
  * polarization       (polarization) <U1 16B 'I' 'Q' 'U' 'V'
  * l                  (l) float64 2kB 6.06e-05 6.012e-05 ... -6.012e-05
  * m                  (m) float64 2kB -6.06e-05 -6.012e-05 ... 6.012e-05
  * beam_params_label  (beam_params_label) <U5 60B 'major' 'minor' 'pa'
    velocity           (frequency) float64 40B 2.216e+05 6.54e+04 ... -4.03e+05
    right_ascension    (l, m) float64 500kB 3.539 3.539 3.539 ... 3.539 3.539
    declination        (l, m) float64 500kB 0.5324 0.5324 ... 0.5325 0.5325
Data variables:
    SKY                (time, frequency, polarization, l, m) float32 5MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    BEAM_FIT_PARAMS    (time, frequency, polarization, beam_params_label) float64 480B dask.array<chunksize=(1, 5, 4, 3), meta=np.ndarray>
    MASK_SKY           (time, frequency, polarization, l, m) bool 1MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
Attributes:
    coordinate_system_info:  {'reference_direction': {'attrs': {'frame': 'fk5...
    type:                    image
    data_groups:             {'base': {'beam_fit_params': 'BEAM_FIT_PARAMS', ...

In [22]:
#Add a new data group with improved image

img_xds["SKY_IMPROVED"] = img_xds["SKY"] * 1.1  # Example improvement: scale the image data


img_xds.xr_img.add_data_group(
        new_data_group_name = "improved",
        new_data_group = {
            "sky": "SKY_IMPROVED",
            "date": "2024-06-10",
            "description": "This data group contains improved image.",
        },
        data_group_dv_shared_with="base"
)

img_xds

<xarray.Dataset> Size: 12MB
Dimensions:            (time: 1, frequency: 5, polarization: 4, l: 250, m: 250,
                        beam_params_label: 3)
Coordinates:
  * time               (time) float64 8B 5.684e+04
  * frequency          (frequency) float64 40B 2.398e+11 2.399e+11 ... 2.403e+11
  * polarization       (polarization) <U1 16B 'I' 'Q' 'U' 'V'
  * l                  (l) float64 2kB 6.06e-05 6.012e-05 ... -6.012e-05
  * m                  (m) float64 2kB -6.06e-05 -6.012e-05 ... 6.012e-05
  * beam_params_label  (beam_params_label) <U5 60B 'major' 'minor' 'pa'
    velocity           (frequency) float64 40B 2.216e+05 6.54e+04 ... -4.03e+05
    right_ascension    (l, m) float64 500kB 3.539 3.539 3.539 ... 3.539 3.539
    declination        (l, m) float64 500kB 0.5324 0.5324 ... 0.5325 0.5325
Data variables:
    SKY                (time, frequency, polarization, l, m) float32 5MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    BEAM_FIT_PARAMS    (time, frequency, polarization, beam_params_label) float64 480B dask.array<chunksize=(1, 5, 4, 3), meta=np.ndarray>
    MASK_SKY           (time, frequency, polarization, l, m) bool 1MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    SKY_IMPROVED       (time, frequency, polarization, l, m) float32 5MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
Attributes:
    coordinate_system_info:  {'reference_direction': {'attrs': {'frame': 'fk5...
    type:                    image
    data_groups:             {'base': {'beam_fit_params': 'BEAM_FIT_PARAMS', ...

In [23]:
display(HTML(dict_to_html(img_xds.attrs["data_groups"])))

In [24]:
#Select the improved data group

img_improved_xds = img_xds.xr_img.sel(data_group_name="improved")
img_improved_xds

<xarray.Dataset> Size: 7MB
Dimensions:            (time: 1, frequency: 5, polarization: 4, l: 250, m: 250,
                        beam_params_label: 3)
Coordinates:
  * time               (time) float64 8B 5.684e+04
  * frequency          (frequency) float64 40B 2.398e+11 2.399e+11 ... 2.403e+11
  * polarization       (polarization) <U1 16B 'I' 'Q' 'U' 'V'
  * l                  (l) float64 2kB 6.06e-05 6.012e-05 ... -6.012e-05
  * m                  (m) float64 2kB -6.06e-05 -6.012e-05 ... 6.012e-05
  * beam_params_label  (beam_params_label) <U5 60B 'major' 'minor' 'pa'
    velocity           (frequency) float64 40B 2.216e+05 6.54e+04 ... -4.03e+05
    right_ascension    (l, m) float64 500kB 3.539 3.539 3.539 ... 3.539 3.539
    declination        (l, m) float64 500kB 0.5324 0.5324 ... 0.5325 0.5325
Data variables:
    BEAM_FIT_PARAMS    (time, frequency, polarization, beam_params_label) float64 480B dask.array<chunksize=(1, 5, 4, 3), meta=np.ndarray>
    MASK_SKY           (time, frequency, polarization, l, m) bool 1MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    SKY_IMPROVED       (time, frequency, polarization, l, m) float32 5MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
Attributes:
    coordinate_system_info:  {'reference_direction': {'attrs': {'frame': 'fk5...
    type:                    image
    data_groups:             {'improved': {'sky': 'SKY_IMPROVED', 'mask_sky':...

In [25]:
img_xds

<xarray.Dataset> Size: 12MB
Dimensions:            (time: 1, frequency: 5, polarization: 4, l: 250, m: 250,
                        beam_params_label: 3)
Coordinates:
  * time               (time) float64 8B 5.684e+04
  * frequency          (frequency) float64 40B 2.398e+11 2.399e+11 ... 2.403e+11
  * polarization       (polarization) <U1 16B 'I' 'Q' 'U' 'V'
  * l                  (l) float64 2kB 6.06e-05 6.012e-05 ... -6.012e-05
  * m                  (m) float64 2kB -6.06e-05 -6.012e-05 ... 6.012e-05
  * beam_params_label  (beam_params_label) <U5 60B 'major' 'minor' 'pa'
    velocity           (frequency) float64 40B 2.216e+05 6.54e+04 ... -4.03e+05
    right_ascension    (l, m) float64 500kB 3.539 3.539 3.539 ... 3.539 3.539
    declination        (l, m) float64 500kB 0.5324 0.5324 ... 0.5325 0.5325
Data variables:
    SKY                (time, frequency, polarization, l, m) float32 5MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    BEAM_FIT_PARAMS    (time, frequency, polarization, beam_params_label) float64 480B dask.array<chunksize=(1, 5, 4, 3), meta=np.ndarray>
    MASK_SKY           (time, frequency, polarization, l, m) bool 1MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
    SKY_IMPROVED       (time, frequency, polarization, l, m) float32 5MB dask.array<chunksize=(1, 5, 4, 250, 250), meta=np.ndarray>
Attributes:
    coordinate_system_info:  {'reference_direction': {'attrs': {'frame': 'fk5...
    type:                    image
    data_groups:             {'base': {'beam_fit_params': 'BEAM_FIT_PARAMS', ...